# Random Forest

### Load Modules

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

### Load Dataset - only traditional features

In [3]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [4]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 162), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [5]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [6]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [7]:
# Model
model_name = 'RF_trad'

In [8]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

model_ix = 0
for depth in [1, 2, 3, 4, 5]:
    for n_features in [3, 5, 10, 20, 30]:
        print("•", model_ix, end=' ')
                    
        for FOLD in range(K_FOLDs):
            VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
            TRAI = [x for x in YEARS if x not in VALI]
            
            # Reset seeds
            np.random.seed(0)
            random.seed(0)
            
            # Create model
            RF = RandomForestRegressor(max_depth=depth, max_features=n_features, n_estimators=300, n_jobs=-1, random_state=0)

            # Fit the model
            RF.fit(X_trai.loc[TRAI], Y_trai.loc[TRAI].values.ravel()) 

            # Calculate validation predictions
            val_preds = RF.predict(X_trai.loc[VALI])
            Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = val_preds
            val_loss = mean_squared_error(Y_trai.loc[VALI], val_preds)
                        
            # Calculate predictions for test data, if FOLD = 0
            if FOLD==0:
                Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = RF.predict(X_test)
            
            # Append results
            results.append({
                'model_ix'  :model_ix,
                'depth'     :depth,
                'n_features':n_features,
                'fold'      :FOLD,
                'val_loss'  :val_loss
            })
            
        model_ix += 1

• 0 • 1 • 2 • 3 • 4 • 5 • 6 • 7 • 8 • 9 • 10 • 11 • 12 • 13 • 14 • 15 • 16 • 17 • 18 • 19 • 20 • 21 • 22 • 23 • 24 

In [9]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  RF_trad_0  RF_trad_1  RF_trad_2  RF_trad_3  \
year YM      permno                                                         
2007 2007-01 10025  -0.031894   0.009980   0.010186   0.010410   0.010563   
             10026  -0.042317   0.009852   0.009787   0.009870   0.009863   
             10042  -0.125751   0.004409   0.003375  -0.000117  -0.001977   
             10078  -0.080607   0.009789   0.009939   0.010015   0.011117   
             10104  -0.046341   0.010561   0.010726   0.010965   0.011600   
...                       ...        ...        ...        ...        ...   
2018 2018-12 93420   0.086508   0.004501   0.003837   0.003363   0.002623   
             93422   0.466817   0.003331   0.001816  -0.000481  -0.002547   
             93423   0.105036   0.006076   0.006122   0.006643   0.007058   
             93429  -0.048712   0.004399   0.003569   0.002067   0.001115   
             93436  -0.079564   0.004393   0.004240   0.003913   0.003087   

                     RF_trad_4  RF_trad_5  RF_trad_6  RF_trad_7  RF_trad_8  \
year YM      permno                                                          
2007 2007-01 10025    0.010585   0.010116   0.010467   0.010881   0.010997   
             10026    0.009809   0.010083   0.010175   0.010012   0.009830   
             10042   -0.003914   0.000788  -0.002270  -0.004586  -0.006489   
             10078    0.011245   0.009743   0.010479   0.011006   0.012124   
             10104    0.011572   0.011041   0.011591   0.011762   0.012585   
...                        ...        ...        ...        ...        ...   
2018 2018-12 93420    0.002096   0.002948   0.001716   0.001286   0.000255   
             93422   -0.005363   0.000447  -0.002800  -0.006348  -0.010774   
             93423    0.007257   0.006503   0.006757   0.007495   0.008387   
             93429   -0.000611   0.003702   0.002452  -0.000158  -0.002485   
             93436    0.002723   0.003644   0.003401   0.002598   0.002405   

                     ...  RF_trad_15  RF_trad_16  RF_trad_17  RF_trad_18  \
year YM      permno  ...                                                   
2007 2007-01 10025   ...    0.010843    0.011173    0.011565    0.012079   
             10026   ...    0.010465    0.010635    0.010037    0.010312   
             10042   ...   -0.009053   -0.011067   -0.015400   -0.010962   
             10078   ...    0.010105    0.011357    0.012118    0.013780   
             10104   ...    0.011892    0.012619    0.012871    0.013843   
...                  ...         ...         ...         ...         ...   
2018 2018-12 93420   ...    0.000080   -0.001494   -0.002074   -0.005379   
             93422   ...   -0.005676   -0.009899   -0.015556   -0.023052   
             93423   ...    0.007032    0.007515    0.008986    0.009966   
             93429   ...    0.002208   -0.001076   -0.005245   -0.008669   
             93436   ...    0.002890    0.002008    0.000861   -0.000994   

                     RF_trad_19  RF_trad_20  RF_trad_21  RF_trad_22  \
year YM      permno                                                   
2007 2007-01 10025     0.011722    0.011960    0.011344    0.011931   
             10026     0.010581    0.010232    0.010596    0.010707   
             10042    -0.005561   -0.017465   -0.017271   -0.023793   
             10078     0.014594    0.010366    0.012245    0.012519   
             10104     0.014746    0.012134    0.012809    0.013193   
...                         ...         ...         ...         ...   
2018 2018-12 93420    -0.006821   -0.003122   -0.006040   -0.007283   
             93422    -0.036334   -0.010564   -0.016274   -0.022991   
             93423     0.010098    0.007671    0.008118    0.008858   
             93429    -0.012969   -0.000029   -0.003157   -0.007004   
             93436    -0.002959    0.001974    0.002556    0.000611   

                     RF_trad_23  RF_trad_24  
year YM      permno                          
2

In [10]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  RF_trad_0  RF_trad_1  RF_trad_2  RF_trad_3  \
year YM      permno                                                         
2019 2019-01 10026   0.004225   0.010195   0.010303   0.010519   0.010464   
             10104   0.036026   0.010573   0.010718   0.010965   0.011600   
             10107   0.075381   0.010239   0.010339   0.010571   0.010624   
             10138   0.072777   0.010423   0.010726   0.010995   0.011600   
             10145   0.076596   0.010649   0.010768   0.010995   0.011600   
...                       ...        ...        ...        ...        ...   
2021 2021-12 93304  -0.096386   0.010508   0.010764   0.011020   0.011600   
             93373  -0.019481   0.007664   0.005646   0.001646  -0.001626   
             93374  -0.047552   0.010251   0.010393   0.010627   0.010624   
             93423  -0.072569   0.011005   0.010878   0.010445   0.010457   
             93436  -0.113609   0.009779   0.009962   0.010083   0.010177   

                     RF_trad_4  RF_trad_5  RF_trad_6  RF_trad_7  RF_trad_8  \
year YM      permno                                                          
2019 2019-01 10026    0.010537   0.010562   0.010811   0.010908   0.010722   
             10104    0.011572   0.011011   0.011553   0.012141   0.014398   
             10107    0.010647   0.010668   0.010755   0.010913   0.010822   
             10138    0.011572   0.010847   0.011565   0.012138   0.014564   
             10145    0.011572   0.011190   0.011727   0.012244   0.014564   
...                        ...        ...        ...        ...        ...   
2021 2021-12 93304    0.011572   0.010897   0.011679   0.011975   0.012857   
             93373   -0.004752   0.008213   0.004188  -0.000760  -0.005838   
             93374    0.010647   0.010786   0.010936   0.011207   0.011147   
             93423    0.010563   0.012709   0.012811   0.013549   0.012677   
             93436    0.010354   0.010048   0.010264   0.010668   0.011672   

                     ...  RF_trad_15  RF_trad_16  RF_trad_17  RF_trad_18  \
year YM      permno  ...                                                   
2019 2019-01 10026   ...    0.011061    0.011298    0.010933    0.011026   
             10104   ...    0.011862    0.013405    0.013083    0.016162   
             10107   ...    0.011173    0.011302    0.010963    0.010816   
             10138   ...    0.011512    0.012908    0.012621    0.017255   
             10145   ...    0.012066    0.013566    0.013012    0.016348   
...                  ...         ...         ...         ...         ...   
2021 2021-12 93304   ...    0.011682    0.012581    0.013036    0.013815   
             93373   ...    0.006628    0.000360   -0.003552   -0.007311   
             93374   ...    0.011460    0.011676    0.011875    0.012046   
             93423   ...    0.015640    0.015312    0.018214    0.017518   
             93436   ...    0.010777    0.010938    0.013112    0.014796   

                     RF_trad_19  RF_trad_20  RF_trad_21  RF_trad_22  \
year YM      permno                                                   
2019 2019-01 10026     0.011006    0.011245    0.011640    0.011253   
             10104     0.020198    0.012000    0.013224    0.014994   
             10107     0.010822    0.011424    0.011431    0.010994   
             10138     0.020043    0.011694    0.013270    0.015061   
             10145     0.020619    0.012071    0.013580    0.015251   
...                         ...         ...         ...         ...   
2021 2021-12 93304     0.014024    0.011994    0.012961    0.013934   
             93373    -0.010318    0.005722   -0.002128   -0.003201   
             93374     0.012046    0.011762    0.011893    0.012197   
             93423     0.014922    0.012035    0.015827    0.016811   
             93436     0.016858    0.010862    0.012188    0.013891   

                     RF_trad_23  RF_trad_24  
year YM      permno                          
2

In [11]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'depth', 'n_features']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['val_loss']].head(20)) * 100

,,,val_loss
model_ix,depth,n_features,
20,5,3,12.932000
15,4,3,12.932201
21,5,5,12.932604
10,3,3,12.933881
11,3,5,12.934224
5,2,3,12.934413
6,2,5,12.934500
16,4,5,12.934646
7,2,10,12.935458


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [12]:
# Model
model_name = 'RF_trad'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_depth = list(pd.unique(table['depth']))
list_n_features = list(pd.unique(table['n_features']))

In [13]:
# Loop over hyperparameter combinations and calculate SR
val_results = []
hyper_comb = 0
for depth in list_depth:
    for n_features in list_n_features:
        # Get relevant model_ix
        model_ix = table[(table['depth'] == depth) & (table['n_features'] == n_features)]
        model_ix = model_ix['model_ix']
        # Select relevant returns
        val_ret = val[['%s_%d'%(model_name, model_ix)]]
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'depth'     :depth,
            'n_features':n_features,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1  

# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))        

In [14]:
# Select best hyperparemeters (max. SR_LS)
list_depth = list(pd.unique(table['depth']))
list_n_features = list(pd.unique(table['n_features']))

val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['depth', 'n_features']).sort_values(['SR_LS'], ascending=False)
depth_opt, n_features_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal depth:     ', depth_opt)
print('Optimal n_features:', n_features_opt)
val_results

Optimal depth:      3
Optimal n_features: 5


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
depth n_features                                                         
3     5                   11          11  0.006305  0.681681  0.745438   
5     3                   20          20 -0.010235  0.629593  0.732495   
      5                   21          21  0.009466  0.649401  0.727736   
2     3                    5           5  0.006336  0.686192  0.711527   
4     3                   15          15 -0.003366  0.638952  0.705133   
3     3                   10          10  0.001964  0.646196  0.704012   
5     30                  24          24  0.099007  0.588739  0.701542   
4     5                   16          16  0.009297  0.644273  0.684499   
      20                  18          18  0.080525  0.607924  0.683666   
      10                  17          17  0.044557  0.645933  0.683322   
2     5                    6           6  0.006318  0.663200  0.680036   
1     3                    0           0  0.011964  0.646135  0.668026   
5     20                  23          23  0.067477  0.569050  0.650867   
1     5                    1           1  0.033314  0.691768  0.644937   
2     10                   7           7  0.048718  0.688043  0.643344   
5     10                  22          22  0.043516  0.605319  0.642202   
1     10                   2           2  0.063737  0.678528  0.631620   
4     30                  19          19  0.100396  0.551250  0.615246   
3     30                  14          14  0.107885  0.584662  0.609455   
      10                  12          12  0.057546  0.644072  0.596791   
2     30                   9           9  0.081539  0.609472  0.589273   
1     20                   3           3  0.075653  0.567961  0.579738   
      30                   4           4  0.091003  0.558243  0.560675   
3     20                  13          13  0.097897  0.568713  0.553318   
2     20                   8           8  0.077310  0.596249  0.548019   

                   Mean_Q1   Mean_Q5   Mean_LS  
depth n_features                                
3     5           0.000144  0.009504  0.009360  
5     3          -0.000235  0.008887  0.009122  
      5           0.000217  0.009253  0.009036  
2     3           0.000147  0.009389  0.009242  
4     3          -0.000078  0.008970  0.009049  
3     3           0.000045  0.009001  0.008956  
5     30          0.002188  0.009489  0.007300  
4     5           0.000214  0.008971  0.008756  
      20          0.001816  0.009420  0.007603  
      10          0.001021  0.009362  0.008340  
2     5           0.000147  0.009021  0.008875  
1     3           0.000274  0.008826  0.008552  
5     20          0.001516  0.008851  0.007335  
1     5           0.000762  0.009152  0.008390  
2     10          0.001125  0.009444  0.008319  
5     10          0.000993  0.008805  0.007812  
1     10          0.001453  0.009346  0.007893  
4     30          0.002226  0.008797  0.006572  
3     30          0.002403  0.009058  0.006656  
      10          0.001320  0.008981  0.007661  
2     30          0.001853  0.008858  0.007004  
1     20          0.001738  0.008454  0.006716  
      30          0.002099  0.008450  0.006351  
3     20          0.002201  0.008628  0.006427  
2     20          0.001771  0.008608  0.006837

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [15]:
# Get relevant model_ix
model_ix = table[(table['depth'] == depth_opt) & (table['n_features'] == n_features_opt)]
model_ix = model_ix['model_ix']

# Select relevant returns
test_ret = test.copy()
test_ret['y_pred'] = test[['%s_%d'%(model_name, model_ix)]]

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)